In [1]:
# ==================
# Library
# ==================
import warnings
warnings.simplefilter('ignore')
import math
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
import datetime
import gc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold,GroupKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from contextlib import contextmanager
import logging
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import log_loss
import sys
import time
import feather
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import pickle
%matplotlib inline
from sklearn.metrics import average_precision_score

sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [2]:
# ==================
# Constant
# ==================
ex = "009"
TRAIN_PATH = "../input/train.csv"
TEST_PATH = "../input/test.csv"
USER_PATH = "../input/user_x_anime.csv"
SUB_PATH = "../input/sample_submission.csv"
SAVE_OOF_PATH = f"../output/exp/ex{ex}_oof.npy"
SAVE_TEST_SUB_PATH = f"../output/exp/ex{ex}_test_sub.csv"
LOGGER_PATH = f"../output/exp/ex_{ex}.txt"

In [3]:

# ===============
# Settings
# ===============

SEED = 0
N_SPLITS = 5
SHUFFLE = True
LGBM_PARAMS = {'num_leaves': 32,
               'min_data_in_leaf': 64,
               'objective': 'regression',
               'max_depth': -1,
               'learning_rate': 0.05,
               "boosting": "gbdt",
               "bagging_freq": 1,
               "bagging_fraction": 0.8,
               "bagging_seed": SEED,
               "verbosity": -1,
              'reg_alpha': 0.1,
              'reg_lambda': 0.3,
              'colsample_bytree': 0.7,
              'metric':"rmse",
              'num_threads':32,
         }

LGBM_FIT_PARAMS = {
    'num_boost_round': 10000,
    'early_stopping_rounds': 200,
    'verbose_eval': 200,
}

load_feature = ["../output/fe/fe001.feather",
                "../output/fe/fe002.feather",
                "../output/fe/fe003.feather",
                "../output/fe/fe004.feather",
                "../output/fe/fe005.feather",
                "../output/fe/fe006.feather",
                "../output/fe/fe007.feather",
                "../output/fe/fe009.feather",]

In [4]:
# ====================
# Function
# ====================

def calc_loss(y_true, y_pred):
    return  np.sqrt(mean_squared_error(y_true, y_pred))


@contextmanager
def timer(name):
    t0 = time.time()
    yield 
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')

In [5]:
LOGGER = logging.getLogger()
FORMATTER = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
setup_logger(out_file=LOGGER_PATH)

2021-08-17 11:55:47,361 - INFO - logger set up


<RootLogger root (DEBUG)>

In [6]:
# ====================
# Main
# ====================
train_raw = pd.read_csv(TRAIN_PATH)
y = train_raw["Score"]
df = None
for i in load_feature:
    if df is not None:
        _df = pd.read_feather(i)
        df  = pd.concat([df, _df], axis=1)
    else:
        df = pd.read_feather(i)
train = df.iloc[:len(train_raw)]
test = df.iloc[len(train_raw):].reset_index(drop=True)

In [7]:
y

0       8.78
1       8.06
2       7.90
3       6.38
4       8.17
        ... 
4995    7.23
4996    6.54
4997    6.51
4998    5.92
4999    6.83
Name: Score, Length: 5000, dtype: float64

In [8]:
with timer("lightgbm"):
    kf = KFold(n_splits=N_SPLITS,random_state=SEED, shuffle=SHUFFLE)
    y_oof = np.empty([len(train),])
    y_test = []
    drop_cols = []
    features = list(train.columns)
    features = [i for i in features if i not in drop_cols]
    feature_importances = pd.DataFrame()
    categorical_features = []
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train,y)):
        print('Fold {}'.format(fold + 1))
        with timer(f"fold {fold}"):
            x_train, y_train = train.iloc[train_idx][features], y.iloc[train_idx]
            x_val, y_val =train.iloc[valid_idx][features], y.iloc[valid_idx]
            print("train:",len(x_train))

            y_pred_valid, y_pred_test, valid_loss, importances, best_iter,_ = train_lgbm(
                        x_train, y_train, x_val, y_val,test[features],
                        categorical_features=categorical_features,
                        feature_name=features,
                        fold_id=fold,
                        lgb_params=LGBM_PARAMS,
                        fit_params=LGBM_FIT_PARAMS,
                        loss_func=calc_loss,
                        calc_importances=True
                    )

            y_oof[valid_idx] = y_pred_valid
            score = calc_loss(y[valid_idx], y_pred_valid)
            LOGGER.info(f'Fold{fold}:CV={score}')
            y_test.append(y_pred_test)
            feature_importances = pd.concat([feature_importances, importances], axis=0, sort=False)

    score = calc_loss(y, y_oof)
    np.save(SAVE_OOF_PATH, y_oof)
    LOGGER.info(f'CV={score}')

Fold 1
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.258653	valid_1's rmse: 0.380074
[400]	training's rmse: 0.186016	valid_1's rmse: 0.372625
[600]	training's rmse: 0.143955	valid_1's rmse: 0.371192
Early stopping, best iteration is:
[518]	training's rmse: 0.159032	valid_1's rmse: 0.370564


2021-08-17 11:55:53,123 - INFO - Fold0:CV=0.3705643720216009
2021-08-17 11:55:53,130 - INFO - [fold 0] done in 5 s


Fold 2
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.254909	valid_1's rmse: 0.380569
[400]	training's rmse: 0.183182	valid_1's rmse: 0.371504
[600]	training's rmse: 0.14135	valid_1's rmse: 0.370053
[800]	training's rmse: 0.113427	valid_1's rmse: 0.37028
[1000]	training's rmse: 0.0932725	valid_1's rmse: 0.369549
[1200]	training's rmse: 0.0784042	valid_1's rmse: 0.369441
[1400]	training's rmse: 0.0668475	valid_1's rmse: 0.369832
Early stopping, best iteration is:
[1203]	training's rmse: 0.0782016	valid_1's rmse: 0.369428


2021-08-17 11:56:05,492 - INFO - Fold1:CV=0.36942781943664554
2021-08-17 11:56:05,499 - INFO - [fold 1] done in 12 s


Fold 3
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.257227	valid_1's rmse: 0.392204
[400]	training's rmse: 0.185547	valid_1's rmse: 0.38077
[600]	training's rmse: 0.144217	valid_1's rmse: 0.378226
[800]	training's rmse: 0.116242	valid_1's rmse: 0.377519
Early stopping, best iteration is:
[755]	training's rmse: 0.121709	valid_1's rmse: 0.377196


2021-08-17 11:56:14,218 - INFO - Fold2:CV=0.3771957408733036
2021-08-17 11:56:14,226 - INFO - [fold 2] done in 9 s


Fold 4
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.261636	valid_1's rmse: 0.378879
[400]	training's rmse: 0.188253	valid_1's rmse: 0.370123


2021-08-17 11:56:19,897 - INFO - Fold3:CV=0.36993855909904033
2021-08-17 11:56:19,904 - INFO - [fold 3] done in 6 s


Early stopping, best iteration is:
[396]	training's rmse: 0.189383	valid_1's rmse: 0.369939
Fold 5
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.254339	valid_1's rmse: 0.403776
[400]	training's rmse: 0.182224	valid_1's rmse: 0.393433
[600]	training's rmse: 0.140414	valid_1's rmse: 0.389589
[800]	training's rmse: 0.112245	valid_1's rmse: 0.388671
[1000]	training's rmse: 0.0919407	valid_1's rmse: 0.388137
Early stopping, best iteration is:
[919]	training's rmse: 0.0993645	valid_1's rmse: 0.38783


2021-08-17 11:56:30,818 - INFO - Fold4:CV=0.38783014658735027
2021-08-17 11:56:30,837 - INFO - [fold 4] done in 11 s
2021-08-17 11:56:30,852 - INFO - CV=0.3750568635896904
2021-08-17 11:56:30,855 - INFO - [lightgbm] done in 43 s


In [9]:
y_test_sub =  np.mean(y_test,axis=0)
sub = pd.read_csv(SUB_PATH)

In [10]:
sub["Score"] = y_test_sub
sub.to_csv(SAVE_TEST_SUB_PATH,index=False)

In [13]:
feature_importances.groupby(by="feature").mean().sort_values(by="gain",ascending=False).iloc[:10]

,gain,split,fold
feature,,,
Favorites,8095.513708,744.8,2.0
Plan to Watch,2464.216153,883.4,2.0
Members,2318.057492,712.2,2.0
Dropped,1844.423078,1610.6,2.0
On-Hold,991.671600,1107.4,2.0
Duration,933.223637,929.2,2.0
anime_id_rate_mean,930.781428,1200.4,2.0
anime_id_rate_std,920.863884,1868.8,2.0
Completed,758.454763,960.6,2.0
